<a href="https://colab.research.google.com/github/Kvnhooman/AAI520_Final-Project_Group5/blob/tommy-dev-1/Financial_Review_AI_Agents_v9.21.953.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title
**Group 5**

Final Project




___
## Outline


**Core Agent Functions**

* Planning Agent: Plans research steps for stock analysis


* Tool Use Agent: Integrates APIs (Yahoo Finance, SEC EDGAR, News APIs)


* Self-Reflection Agent: Evaluates output quality and iterates


* Learning Agent: Maintains memory across analysis runs


**Workflow Patterns**
* Prompt Chaining: News ingestion → preprocessing → classification → extraction → summarization


* Routing: Directs content to specialist analyzers (earnings, news, market)


* Evaluator-Optimizer: Generates analysis → evaluates quality → refines using feedback

In [ ]:
# Install dependencis
# Attribution: Geek for Geeks tutorial - https://www.geeksforgeeks.org/artificial-intelligence/introduction-to-langchain/

# -- CORE LANGCHAIN AND PLUGINS --
!pip install -U langchain langchain-openai langchain-community langchain-google-genai

# -- LARGE MODEL PROVIDERS/SKILL ADAPTERS --
!pip install -U google-generativeai huggingface_hub openai

# -- COMMON TOOLING AND UTILITIES --
!pip install -U python-dotenv yfinance duckduckgo-search

# -- DUCKDUCKGO SEARCH API WRAPPER --
!pip install -U ddgs

# -- OPTIONAL: For advanced memory (semantic search/vector db) --
!pip install -U faiss-cpu  # For in-memory vector DBs (Lightweight)
# If you want persistent/production memory, add chromadb or qdrant-client

# -- Install LangGraph for advanced agent orchestration --
!pip install -U langgraph

# -- (OPTIONAL) For running Python tool actions securely --
!pip install -U restrictedpython

# Restart the runtime after running this cell if prompted!

  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.7.0
    Uninstalling google-ai-generativelanguage-0.7.0:
      Successfully uninstalled google-ai-generativelanguage-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.12 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
# Import key libraries
import os
import requests
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_react_agent, AgentExecutor, initialize_agent, Tool, AgentType
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool  # newer import for @tool decorator
from langchain.memory import ConversationBufferMemory

In [ ]:
#Set up LLM API Calls
from google.colab import userdata

gemini_key = userdata.get('GEMINI')
hf_key = userdata.get('HF_TOKEN')
openai_key = userdata.get('OPENAI')

In [ ]:
# Setup Gemini to use in Agents
from langchain_google_genai import ChatGoogleGenerativeAI
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=gemini_key
)

In [ ]:
# Test
# Test if Gemini LLM is reachable
try:
    response = llm_gemini.invoke("Say 'GoogleAPI-success-check'")
    print("Gemini API Response:", response)
except Exception as e:
    print("Error reaching Gemini API:", e)


Gemini API Response: content='GoogleAPI-success-check' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []} id='run--82154020-dbca-4c8d-bd08-00f8e180a01e-0'


In [ ]:
'''
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    model="gpt-4o",  # or "gpt-3.5-turbo", "gpt-4", etc.
    openai_api_key=openai_key,  # Your OpenAI API key
    temperature=0.0             # (optional) set as needed
)

In [ ]:
'''
#Test
response = llm_openai.invoke("Explain how neural networks work in 10 words or less.")
print(response.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
'''
!pip install langchain-huggingface huggingface_hub

!pip install langchain_community

In [ ]:
'''
# For LangChain HuggingFace integration
from langchain_huggingface import HuggingFaceEndpoint

llm_hf = HuggingFaceEndpoint(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=hf_key   # Hugging Face API key
)

In [ ]:
'''
# Test
# Test if HF LLM is reachable
try:
    response = llm_hf.invoke("Describe a neural network in 10 words of less'")
    print("Gemini API Response:", response)
except Exception as e:
    print("Error reaching HuggingFace API:", e)

Error reaching HuggingFace API: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.


## Planning Agent
Plans research steps for stock analysis

0. Evaluate user input, infer that they want to know about a stock, or up to three stocks. They may or may not write the stock symbol.
1. Ask, what are the top things to know when analyzing any stock? (Shortcut by hitting top 3 analysis compaines? (Morningstar, Moody, Bloomberg?)
2. Research those things for a limited amount of tokens (how do we limit web search API counts? We could somehow hit an API like wikipedia or something similar if it exists)
3. Ingest news
4. Consider second order effects that might impact this stock or industry, serach news for that
5. Evaluate, compare and contrast and then summarize.
6. Tell tools agent which tools to use based on plan. Pass instructions of where to look and likely tools to use to Tools Agent. Also plan expected output,
- Stock Name
- Stock Symbol
- Value
- Category (growth or stock)
- Recommended strategy (buy, hold, sell)
- Brief pros and cons, including key organization news as well as indsutry trends.

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

simple_prompt = PromptTemplate.from_template(
    "You are a planning agent. Give a plan for: {input}"
)
chain = LLMChain(prompt=simple_prompt, llm=llm_gemini)

response = chain.invoke({"input": "Analyze the SPY stock ticker"})
print("Simple LLMChain response:", response)


/tmp/ipython-input-1292110460.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=simple_prompt, llm=llm_gemini)


Simple LLMChain response: {'input': 'Analyze the SPY stock ticker', 'text': "## Plan: Analyze SPY Stock Ticker\n\nThis plan outlines the steps to analyze the SPY (SPDR S&P 500 ETF Trust) stock ticker, focusing on a comprehensive approach encompassing technical, fundamental, and sentiment analysis.  The plan is adaptable to different time horizons (short-term trading, long-term investment).\n\n**Phase 1: Data Acquisition & Preparation (1-2 days)**\n\n1. **Identify Data Sources:**\n    * **Financial Data Providers:**  Yahoo Finance, Google Finance, Alpha Vantage, Tiingo, Refinitiv Eikon (for professional-grade data). Choose one or more based on data availability, cost, and desired features.\n    * **Brokerage Platforms:** If you use a brokerage account, leverage its charting and data tools.\n    * **Alternative Data:** Consider incorporating sentiment data from social media (e.g., Twitter sentiment analysis), news articles (e.g., using natural language processing), and economic indicator

In [ ]:
'''
# Gemini Approach
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Basic chain-of-thought agent, no true ReAct logic:
prompt = PromptTemplate.from_template(
    """
    You are an intelligent planner. For the following task, think step by step and devise a thorough plan:
    Task: {input}
    """)
chain = prompt | llm_gemini

result = chain.invoke({"input": "Analyze the SPY stock ticker"})
print(result)

In [ ]:
# ReAct Planning agent example from Geek for Geeks
# Define planning prompt
PLANNER_PROMPT = """
You are a thoughtful agent that reasons and acts in this format:

Thought: you should always think before acting
Action: create a step-by-step plan to analyze the stock
Action Input: the input to the action
Observation: the result of the action

(Repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the best plan
Final Answer: the final plan to provide to tool agent is

Question: {input}
{agent_scratchpad}
"""

@tool
def echo(text: str) -> str:
    """Return the input text as the output."""
    return text

tools = [echo]

# Create planner agent
planner_agent = create_react_agent(
    model=llm_gemini,
    tools=tools,
    prompt=PLANNER_PROMPT
)


# --- Test input for the agent ---

print("Sending to agent:", {"input": "Analyze the SPY stock ticker"})
print(llm_gemini.invoke("test: does this call work?"))

response = planner_agent.invoke({"input": "Analyze the SPY stock ticker"})

# Helper to extract the response text (if needed)
try:
    output_text = response['output']    # Many agents return {'output': ...}
except (TypeError, KeyError):
    output_text = str(response)         # Fall back to direct string conversion

# You can use a tokenizer for exact token count,
# but most practical, use whitespace split for demonstration:
words = output_text.split()
first_100 = " ".join(words[:100])

print("First 100 words/tokens:\n")
print(first_100)

Sending to agent: {'input': 'Analyze the SPY stock ticker'}
content='Please provide the code snippet or context of the "call" you\'re referring to.  I need to see the code to tell you if it works.  For example, is it a function call, a method call, a system call, or something else?  What language is it written in?  What are you expecting it to do?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []} id='run--6e67be2e-9b66-4492-8e73-b93f8f3d87dd-0'


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents: contents is not specified


## Tool Agent
Integrates APIs (Yahoo Finance, SEC EDGAR, News APIs)

List of Tools:
- Web Search
- API call - Yahoo Finance

### Tool Functions


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Web search tool
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="Web Search",
    func=search.run,
    description="Search the web and return relevant results"
)

# Yahoo Finance tool
import yfinance as yf
def get_stock_price(ticker: str) -> str:
    stock = yf.Ticker(ticker)
    price = stock.info['regularMarketPrice']
    return f"{ticker} price is {price}"

yahoo_tool = Tool(
    name="Yahoo Finance",
    func=get_stock_price,
    description="Get the latest stock price for a given ticker symbol"
)

# OTHER APIs & TOOLS HERE IN THE FUTURE

# ]Python REPL tool
def python_eval(code: str) -> str:
    try:
        return str(eval(code))
    except Exception as e:
        return f"Error: {e}"

python_tool = Tool(
    name="Python REPL",
    func=python_eval,
    description="Run Python code and return the result"
)

### Tools Agent Function

In [ ]:
# Tool execution agent

REACT_TOOL_PROMPT = """
You are an expert analyst and researcher. Using ONLY the tools provided, thoughtfully follow these steps:

1. Perform any pre-processing or data gathering needed.
2. Use the appropriate tool(s) to answer or analyze the user's request.
3. (If analysis requires multiple steps) Chain tools or aggregate intermediate results.
4. Summarize your findings in a concise and useful way for the end user.

At each stage, state your Thought, Action, Action Input, and Observation as needed.

Reminder: Do NOT make up information—use only direct tool outputs.

Final Answer: A succinct, well-organized summary of findings appropriate for the user.
Question: {input}
{agent_scratchpad}
"""

tools = [search_tool, yahoo_tool, python_tool]  # Pass tools above to pecialist list here

# Create tool

tools_executor = create_react_agent(
    model=llm_gemini,
    tools=tools,
    prompt=REACT_TOOL_PROMPT
)

## Self Reflection & Evaluation Agent
Evaluates output quality and iterates

In [ ]:
# Define Self Evaluation agent

EVAL_PROMPT = """
You are an expert evaluator. Review the analysis/summary below for:

- Completeness (all important steps/points covered)
- Succinctness (concise, minimal repetition)
- Accuracy (supported by real or tool-sourced information)
- Clarity (well organized, easy to follow)

Give clear, specific suggestions if any improvement is needed.
Return the revised summary/answer if changes are warranted.
Otherwise, state that the answer is adequate.

--- Analysis To Evaluate ---
{input}
"""

self_evaluator = create_react_agent(
    model=llm_gemini, # Or gemini_model if that's your variable
    tools = [],
    prompt=EVAL_PROMPT
)

## Optimization Agent

In [ ]:
# Define optimizer agent

OPTIMIZER_PROMPT = """
You are an optimization agent. Given the evaluator's feedback and the initial summary below,
produce a revised version that fixes any weaknesses cited (completeness, succinctness, accuracy, clarity).

Evaluator Feedback:
{feedback}
Initial Answer:
{answer}
---
Optimized Revised Answer:
"""

optimizer_agent = create_react_agent(
    model=llm_gemini, # Or gemini_model if that's your variable
    tools = [],
    prompt=OPTIMIZER_PROMPT
)

## Learning Agent
Maintains memory across analysis runs

In [ ]:
#Memory approach here, to be done later

## Multiple Agent Setup

In [ ]:
from typing_extensions import TypedDict

class MyStateType(TypedDict):
    user_input: str
    planner_output: str
    tool_result: str
    evaluation: str
    optimization: str

In [ ]:
user_input = "Analyze the SPY stock ticker" #Example user input.

In [ ]:
from typing_extensions import TypedDict

class MyStateType(TypedDict):
    user_input: str
    planner_output: str
    tool_result: str
    evaluation: str
    optimization: str

In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(MyStateType)

# Add agents directly as nodes
builder.add_node("Planner", planner_agent)
builder.add_node("ToolsExecutor", tools_executor)
builder.add_node("Evaluator", self_evaluator)
builder.add_node("Optimizer", optimizer_agent)

# Define the entry point and edges
builder.set_entry_point("Planner")

# Define edges
builder.add_edge("Planner", "ToolsExecutor")
builder.add_edge("ToolsExecutor", "Evaluator")
builder.add_edge("Evaluator", "Optimizer")
builder.add_edge("Optimizer", END)


workflow = builder.compile()

# Example user input
user_input = "Analyze the SPY stock ticker"

# Invoke the workflow
response = workflow.invoke({"user_input": user_input})
print(response)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents: contents is not specified


# EXTRA
___

In [ ]:
#Create a prompt template
template = "Give me 3 career skills that are in high demand in {year}."
prompt_template = PromptTemplate.from_template(template)

In [ ]:
#Clean response and ensure is in string format
chain = prompt_template | llm | StrOutputParser()

In [ ]:
#Invoke the chain to get the results
response = chain.invoke({"year": "2025"})
print("\n Career Skills in 2025:\n", response)

In [ ]:
from langchain.agents import load_tools, initialize_agent

tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What is 25 multiplied by 16?")
print(response)

In [ ]:
#Force responess through parser to enforce data schema in responses for downstream benefits
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schemas = [
    ResponseSchema(name="language",
                   description="Detected language of the text"),
    ResponseSchema(
        name="sentiment", description="Sentiment of the text (positive/negative/neutral)")
]

output_parser = StructuredOutputParser.from_response_schemas(schemas)

format_instructions = output_parser.get_format_instructions()
template = """
Analyze the following text:
{text}

{format_instructions}
"""
prompt = PromptTemplate(template=template, input_variables=[
                        "text"], partial_variables={"format_instructions": format_instructions})

chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run(
    text="I love learning about AI, it makes me very excited.")
parsed = output_parser.parse(response)

print(parsed)

In [ ]:
#Example ReAct agent handling JSON from Geek for Geeks
def echo_tool(input_text: str) -> str:
    """Echo tool that expects JSON and returns it back with 'status=processed'."""
    try:
        data = json.loads(input_text)
        data["status"] = "processed"
        return json.dumps(data, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})


tools = [
    Tool(
        name="EchoJSON",
        func=echo_tool,
        description="Takes a JSON string and returns it back with 'status=processed'."
    )
]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = """Take the following info and return JSON with status:
{"name": "Alice", "age": 25, "city": "New York"}"""

response = agent.run(query)
print("\n Final Response:\n", response)

In [ ]:
#Example Agent + Tools execution from Geek for Geeks
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

tools = [calc_tool, python_tool, pdf_tool, search_tool]

llm = ChatOpenAI(model="gpt-4")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run(
    "Calculate 15*12, run 10+25, and simulate a web search for LangChain.")
print(response)

In [ ]:
'''
# Simple Multiagent System with Real LLMs
import openai
import time

OPENAI_API_KEY = ""  # Replace with actual key
client = openai.OpenAI(api_key=OPENAI_API_KEY)


# BASE AGENT CLASS
class Agent:
    def __init__(self, name, role, model="gpt-3.5-turbo"):
        self.name = name
        self.role = role
        self.model = model
        self.memory = []

    def call_llm(self, prompt):

        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": f"You are {self.name}, a {self.role} agent."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300,
                temperature=0.7
            )
            result = response.choices[0].message.content
            print(f"{self.name}: {result[:60]}...")
            return result
        except Exception as e:
            print(f" API failed for {self.name}: {e}")
            return f"Mock response from {self.name}: {prompt[:50]}..."

    def process(self, task):

        prompt = f"As a {self.role}, {task}"
        result = self.call_llm(prompt)
        self.memory.append({"task": task, "result": result})
        return result

    def send_to(self, other_agent, message):

        print(f" {self.name} → {other_agent.name}: {message[:40]}...")
        return other_agent.process(f"Process this from {self.name}: {message}")

# SPECIALIZED AGENTS

class Coordinator(Agent):
    def __init__(self):
        super().__init__("Boss", "project coordinator", "gpt-4")
        self.team = []

    def add_agent(self, agent):
        self.team.append(agent)
        print(f"➕ Added {agent.name} to team")

    def delegate_project(self, project):
        print(f"\n COORDINATING PROJECT: {project}")
        plan = self.process(f"Create a plan for: {project}")

        results = {}
        for agent in self.team:
            task = f"Work on project '{project}' using your {agent.role} skills"
            results[agent.name] = agent.process(task)

        return {"plan": plan, "results": results}

class Researcher(Agent):
    def __init__(self):
        super().__init__("Scout", "researcher", "gpt-3.5-turbo")

    def research(self, topic):
        return self.process(f"research and gather information about: {topic}")

class Analyst(Agent):
    def __init__(self):
        super().__init__("Brain", "data analyst", "gpt-4")

    def analyze(self, data):
        return self.process(f"analyze this data and provide insights: {data}")

class Writer(Agent):
    def __init__(self):
        super().__init__("Pen", "content writer", "gpt-3.5-turbo")

    def write_report(self, content):
        return self.process(f"write a professional report based on: {content}")


# MULTIAGENT SYSTEM

class MultiAgentTeam:
    def __init__(self):
        # Create team
        self.coordinator = Coordinator()
        self.researcher = Researcher()
        self.analyst = Analyst()
        self.writer = Writer()

        # Setup team
        self.coordinator.add_agent(self.researcher)
        self.coordinator.add_agent(self.analyst)
        self.coordinator.add_agent(self.writer)

        print(" Multiagent team created!")

    def execute_project(self, project_description):
        print(f"\n EXECUTING PROJECT")
        print("=" * 40)

        # Step 1: Coordinate
        coordination = self.coordinator.delegate_project(project_description)

        # Step 2: Research
        research_data = self.researcher.research(project_description)

        # Step 3: Analyze research
        analysis = self.researcher.send_to(self.analyst, research_data)

        # Step 4: Write report
        final_report = self.analyst.send_to(self.writer, analysis)

        print(f"\n PROJECT COMPLETED!")
        return {
            "coordination": coordination,
            "research": research_data,
            "analysis": analysis,
            "report": final_report
        }

    def show_team_status(self):
        print(f"\n TEAM STATUS")
        agents = [self.coordinator, self.researcher, self.analyst, self.writer]
        for agent in agents:
            print(f"  {agent.name} ({agent.role}): {len(agent.memory)} tasks completed")


# DEMO FUNCTIONS

def demo_basic_multiagent():
    print(" MULTIAGENT SYSTEM DEMO")
    print("=" * 40)

    # Create team
    team = MultiAgentTeam()

    # Execute project
    project = "Analyze AI market trends and create business strategy"
    results = team.execute_project(project)

    # Show status
    team.show_team_status()

    return results

def demo_agent_communication():
    print(f"\n AGENT COMMUNICATION")
    print("-" * 30)

    researcher = Researcher()
    analyst = Analyst()

    # Research → Analysis chain
    research = researcher.research("artificial intelligence trends")
    analysis = researcher.send_to(analyst, research)

    print(" Communication chain completed!")

def demo_different_models():

    print(f"\n DIFFERENT LLM MODELS")
    print("-" * 30)

    fast_agent = Agent("Fast", "assistant", "gpt-3.5-turbo")
    smart_agent = Agent("Smart", "analyst", "gpt-4")

    task = "Evaluate autonomous vehicles"

    fast_result = fast_agent.process(task)
    smart_result = smart_agent.process(task)

    print(" Different models demonstrated!")

# MAIN EXECUTION

def run_multiagent_demo():
    print("MULTIAGENT SYSTEM WITH REAL LLMs")
    print("=" * 50)
    print("  Set your OpenAI API key at the top!")

    try:
        # Main demo
        results = demo_basic_multiagent()

        # Additional demos
        demo_agent_communication()
        demo_different_models()

    except Exception as e:
        print(f" Demo failed: {e}")
        print(" Check your API key!")

if __name__ == "__main__":
    run_multiagent_demo()

MULTIAGENT SYSTEM WITH REAL LLMs
  Set your OpenAI API key at the top!
 MULTIAGENT SYSTEM DEMO
➕ Added Scout to team
➕ Added Brain to team
➕ Added Pen to team
 Multiagent team created!

 EXECUTING PROJECT

 COORDINATING PROJECT: Analyze AI market trends and create business strategy
Boss: Phase 1: Market Research

1.1 Identify Key Research Areas: I...
Scout: As a researcher, I would approach the project 'Analyze AI ma...
Brain: Project: Analyze AI market trends and create business strate...
Pen: Title: Analyzing AI Market Trends and Crafting a Successful ...
Scout: Analyzing AI market trends and creating a business strategy ...
 Scout → Brain: Analyzing AI market trends and creating ...
Brain: Based on the information provided by Scout, the key points t...
 Brain → Pen: Based on the information provided by Sco...
Pen: Here is a summary based on the key points provided by Scout ...

 PROJECT COMPLETED!

 TEAM STATUS
  Boss (project coordinator): 1 tasks completed
  Scout (researcher): 2 

In [ ]:
'''
#Example tool setup in LangChain from Geek for Geeks
from langchain.agents import Tool

def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"


calc_tool = Tool(
    name="Calculator",
    func=calculator,
    description="Evaluate a math expression like 2+2"
)

result = calc_tool.run("125 * 12")
print("Calculator Tool Output:", result)